In [1]:
import json
import os
import glob
import re
from typing import Optional as Opt, Tuple, Union, List
from PIL import Image
import cv2
import io
import base64
import numpy as np
import requests
import hashlib
import pandas as pd

from datetime import datetime
from dateutil import parser
import pytz

import uuid
import warnings

import pydicom
from pydicom.dataset import FileDataset as pydicomFileDataset, FileMetaDataset as pydicomFileMetaDataset
from pydicom import Dataset as pydicomDataset, Sequence, dcmread, dcmwrite
from pydicom.dataelem import DataElement
from pydicom.datadict import dictionary_VR, dictionary_has_tag
from pydicom.uid import UID as pydicomUID, generate_uid as generate_pydicomUID


from pyxnat import Interface, schema

from pathlib import Path, PurePosixPath

import shutil
import tempfile

from src.utilities import LibrarianUtilities, MetaTables, USCentralDateTime, XNATLogin, XNATConnection, ImageHash
from src.xnat_experiment_data import *
from src.xnat_scan_data import *
from src.xnat_resource_data import ORDataIntakeForm


login_info, verbose = { 'Username': 'dmattioli', 'Password': 'PooPoopoopoo123$', 'Url': 'https://rpacs.iibi.uiowa.edu/xnat/' }, True
validated_login = XNATLogin( login_info, verbose=verbose )
connection = XNATConnection( validated_login, stay_connected=True, verbose=verbose )
metatables = MetaTables( validated_login, connection, verbose=verbose )

-- Validated XNATLogin --
	User: dmattioli
	Server: https://rpacs.iibi.uiowa.edu/xnat/

-- XNAT Connection --
Status:		Open
Signed-in as:	dmattioli
Project:	<Project Object> GROK_AHRQ_real `AHRQ-GROK Intraoperative Arthroscopy and Trauma Data` (protected) 0 subject  (owner: domattioli) (created on 2024-07-01 08:35:54.707) https://rpacs.iibi.uiowa.edu/xnat//data/projects/GROK_AHRQ_real?format=html

SUCCESS! -- Loaded metatables from: C:\Users\DMATTI~1\AppData\Local\Temp\tmpef33ua0_\MetaTables.json
	...Metatables successfully populated from XNAT data.
table_name:  SUBJECTS
col in table:  False
col in table:  False
table_name:  IMAGE_HASHES
col in table:  False
col in table:  False
table_name:  SURGEONS
col in table:  True
col in table:  True
col in table:  True

-- MetaTables -- Accessed by: DMATTIOLI
   *Last Modified: 2024-07-01T11:22:30.279138-05:00
	Table: REGISTERED_USERS
	1    DMATTIOLI                                         
	2    GTHOMAS                                          

### May need to re-think the experimentdata class(es) inputs now that the resource data can store meta info like acquisition site, group, datetime, etc

In [2]:
# d = Path( r'R:\Anderson_Colaborations\OR Data Collection\Temporary-Sensitive_Data\Case3_6-11_945_Arthro_Shoulder' )
# source = 'UNIVERSITY_OF_IOWA_HOSPITALS_AND_CLINICS'
# group = 'SHOULDER_ARTHROSCOPY'
# dt = USCentralDateTime( '2024-06-11-063932')
# zip_dest = r'C:\Users\dmattioli\Projects\XNAT-Interact\data\tmp'

intake_form = ORDataIntakeForm( metatables=metatables, login=validated_login )

Is there an OR Data Intake Form available for this procedure?
	Enter "1" for Yes or "2" for No

--- Surgical Procedure Information ---
	(1/29)	Institution Name	--	Please Copy-and-Paste from the following list:	['UNIVERSITY_OF_IOWA_HOSPITALS_AND_CLINICS', 'UNIVERSITY_OF_HOUSTON', 'AMAZON_MECHANICAL_TURK']
Invalid entry for Institution Name! Please enter one of the options listed above

	(2/29)	Type of Orthro Procedure	--	Please enter "1" for Trauma or "2" for Arthro

	(3/29)	Ortho Procedure Name	--	Please select from the following options:
		Enter '1A' for OPEN REDUCTION HIP FRACTURE–DYNAMIC HIP SCREW
		Enter '1B' for OPEN REDUCTION HIP FRACTURE–CANNULATED HIP SCREW
		Enter '1C' for CLOSED REDUCTION HIP FRACTURE–CANNULATED HIP SCREW
		Enter '1D' for PERCUTANEOUS SACROLIAC FIXATION
		Enter '1E' for OPEN AND PERCUTANEOUS PILON FRACTURES
		Enter '1F' for INTRAMEDULLARY NAIL-CMN
		Enter '1G' for INTRAMEDULLARY NAIL-ANTEGRADE FEMORAL
		Enter '1H' for INTRAMEDULLARY NAIL-RETROGRADE FEMORAL
		

In [16]:
in_dir = Path( intake_form.relevant_folder ) # type: ignore -- for all of these intake form fields because they are Opt[str] and thus its possible that theyre None.
group = intake_form.group
acquisition_site = intake_form.acquisition_site
procedure_type = intake_form.ortho_procedure_type
dt = intake_form.datetime

metatables = MetaTables( validated_login, connection, verbose=verbose )
source_images = SourceESVSession( pn=in_dir,
                                         login=validated_login, xnat_connection=connection, metatables=metatables,
                                         acquisition_site=acquisition_site, group=group, datetime=dt, # type: ignore
                                         resource_files=[intake_form])
print( source_images )

SUCCESS! -- Loaded metatables from: C:\Users\DMATTI~1\AppData\Local\Temp\tmp7agjri4g\MetaTables.json
	...Metatables successfully populated from XNAT data.
table_name:  SUBJECTS
col in table:  False
col in table:  False
table_name:  IMAGE_HASHES
col in table:  False
col in table:  False
table_name:  SURGEONS
col in table:  True
col in table:  True
col in table:  True

-- MetaTables -- Accessed by: DMATTIOLI
   *Last Modified: 2024-07-01T11:22:30.279138-05:00
	Table: REGISTERED_USERS
	1    DMATTIOLI                                         
	2    GTHOMAS                                           
	...
	6    JHILL7                                            
	7    EZWILLIAMS                                        

	Table: ACQUISITION_SITES
	1    UNIVERSITY_OF_IOWA_HOSPITALS_AND_CLINICS          
	2    UNIVERSITY_OF_HOUSTON                             
	3    AMAZON_MECHANICAL_TURK                            

	Table: GROUPS
	1    OPEN_REDUCTION_HIP_FRACTURE–DYNAMIC_HIP_SCREW     
	2    OPE

In [20]:
print( source_images.df)

TypeError: 'str' object is not callable

In [19]:
source_images.write( verbose=True )
# source_images.write_publish_catalog_subroutine( schema_prefix_str='esv', verbose=True )

	SUCCESS! --- Added "2_25_219782268944444994491944262331939733572" to table "SUBJECTS"


IndexError: index 0 is out of bounds for axis 0 with size 0

##### Delete all subjects in server

In [5]:
# %run src/delete_contents_of_server.py --server {login_info['Url']} --username {login_info['Username']} --password {login_info['Password']}

https://rpacs.iibi.uiowa.edu/xnat/ dmattioli PooPoopoopoo123$
['202306334', 'AF-OTA', 'domSandBox', 'ExampleProject', 'IOWA_UIA', 'Mouse_Atlas', 'OBL_GROK_SST', 'PAO-JCS', 'SURGSIM', 'TestProject', 'TestProjectID']
['RPACS_S52872', 'RPACS_S52890', 'RPACS_S54228', 'RPACS_S54229', 'RPACS_S54230', 'RPACS_S54231', 'RPACS_S54233', 'RPACS_S54234', 'RPACS_S54235']
Failed to delete subject:  RPACS_S52872
Failed to delete subject:  RPACS_S52890


In [4]:
class MTurkSemanticSegmentation( ScanFile ):
    ''' # Example usage:
    print( MTurkSemanticSegmentation( pd.read_csv( r'...\\data\\examples\\MTurkSemanticSegmentation_Example_File.csv' ) ) )
    '''
    def __init__( self, assignment: pd.Series, metatables: MetaTables ): #to-do: allow for different input types eg batch file data or pulled-from-xnat data
        super().__init__( metatables )  # Call the __init__ method of the base class
        self._validate_input( assignment )
        self._read_image()
        self._extract_target_object_info() #to-do
        self._extract_date_and_time()
        # self._extract_uid_info()
        self._extract_pngImageData()
        self._check_data_validity()

    @property
    def bw( self ) -> np.ndarray:   return self._bw
    
    def _validate_input( self, assignment: pd.Series ):
        assert len( set(self.mturk_batch_col_names) - set(assignment.columns) ) == 0, f"Missing required columns: {set(self.mturk_batch_col_names) - set(assignment.columns)}"
        self._metadata = assignment.loc[0]
        img_s3_url = assignment.loc[0,'Input.image_url']
        assert self.is_s3_url( img_s3_url ), f'Input.image_url column of inputted data series (row) must be an s3 url: {img_s3_url}'
        self._ffn = self.metadata['Input.image_url']
        self._bw, self._acquisition_site = self.image.dummy_image(), 'AMAZON_MECHANICAL_TURK' #to-do: this is copy-pasted from the MetaTables, need to figure out how to query it.

    def _check_data_validity( self ): # need to check that the pnd image *does* exist in metatables
        self._is_valid = self.image.in_img_hash_metatable and not self.is_similar_to_template_image()
    
    def _extract_target_object_info( self ):
        pass

    def _extract_date_and_time( self ):
        self._datetime = USCentralDateTime( self.metadata.loc['SubmitTime'] )

    def _extract_pngImageData( self ):
        pngImageData_index = [i for i, c in enumerate( self.metadata.index.to_list() ) if '.pngImageData' in c]
        self._bw = self.convert_base64_to_np_array( self.metadata.iloc[pngImageData_index[0]] )

     
    def __str__( self ):
        return f'{self.__class__.__name__}:\t{self.ffn}\nIs Valid:\t{self.is_valid}\nAcquisition Site: {self.acquisition_site}\nGroup:\t\t{self.group}\nDatetime:\t{self.datetime}\nUID Info: {self.uid_info}'

In [21]:
def write_acceptable_keywords( metatables: MetaTables ):
    # walk through the 'GROUPS' and 'ACQUISITION_SITES' tables and write the Name for each item as a row in a text file that mimics a compact dataframe
    target_tables = ['GROUPS','ACQUISITION_SITES']
    out_ffn = os.path.join( metatables.doc_dir, 'acceptable_upload_keyword_inputs.txt')
    text_to_write = f'Acceptable Inputs -- Uploading a New Performance\n{"---"*20}\n\n'
    for t_name in target_tables:
        table_items_list = metatables.list_of_all_items_in_table( table_name=t_name )
        text_to_write += f'Key: {t_name}\n'
        for item in table_items_list:
            text_to_write += f'\t- {item}\n'
        text_to_write += '\n'
        
    with open( out_ffn, 'w' ) as f:
        f.write( text_to_write )
    # with open( f'{t_name}.txt', 'w' ) as f:
    #     f.write( f'{t_name}\n' )
    #     f.write( f'NAME\n' )

write_acceptable_keywords( metatables )
    

In [131]:
class OtherDicomSession():
    def __init__( self, batch_ffn: str, login: XNATLogin, xnat_connection: XNATConnection, metatables: MetaTables, object: str, print_out: bool=False ):
        self._batch_ffn = batch_ffn
        self._login = login
        self._xnat_connection = xnat_connection
        self._metatables = metatables
        assert self._metatables.item_exists( table_name='Acquisition_sites', item_name='AMAZON_MECHANICAL_TURK' )
        assert self._metatables.table_exists( 'Segmentation_Objects' )
        assert self._metatables.item_exists( table_name='Segmentation_Objects', item_name=object )
        self._acquisition_site, self._object = 'AMAZON_MECHANICAL_TURK', object

        
        # metatables.add_new_item( table_name='MTURK_BATCH_IDS', item_name='Segmentation', extra_columns_values={'task':'Segmentation'} )


        # attributes that metatables will eventually need to store -- to-do:
        self._object, self._object_type = object, 'BONE'
        self._object_code, self._object_type_code, self._CodingSchemeDesignator = 'T-12760', 'T-32000', 'SRT' # Code for 'Femur', 'Bone', according to copilot gpt
        self._SegmentAlgorithmType, self._SegmentAlgorithmName, self._SegmentationType = 'MANUAL', 'MTURK', 'BINARY'
        
        self._mediastoragesopclassuid, self._transfersyntaxuid = pydicomUID('1.2.840.10008.5.1.1.4'), pydicomUID('1.2.840.10008.1.2') # Basic Grayscale Image Box SOP Class, Implicit VR Little Endian
        self._print_out = print_out    

        # Create a new df with the number of rows equal to the number of groups. create a column for the respective Input.image_url
        self._raw_data = pd.read_csv( self._batch_ffn )
        self._datetime = USCentralDateTime( self._raw_data.loc[0, 'ApprovalTime'] )
        icol = self._get_pngImageData_col()
        self._raw_data['BinaryImages'] = [self.convert_base64_to_np_array( row[icol] ) for i, row in self._raw_data.iterrows()]
        self._grouped_data = self._raw_data.groupby( 'Input.image_url' )
        self._df = pd.DataFrame( index=range( len( self._grouped_data ) ), columns=['SUBJECT_XNAT_UID',
                                                                                    'image_hashes_mt_uid', 'subject_mt_uid',
                                                                                    'FN', 'IS_VALID', 'IMAGE_HASHES', 'BWS' 'DCMDATA', 'UID_INFO', 'FFN'] )
        
        # for each row of self._df compute the ImageHash for the s3url in that row's Image
        self._df['FFN'] = [group for group, data in self._grouped_data]
        self._df['FN'] = [s3url.split('/')[-1] for s3url in self._df['FFN']]
        self._df['IMAGE_HASHES'] = [self._read_image( s3url ) for s3url in self._df['FFN']]
        self._df['UID_INFO'] = self._grouped_data.apply(lambda g: {'HITIDS': g['HITId'].tolist(), 
                                                            'ASSIGNMENTIDS': g['AssignmentId'].tolist(), 
                                                            'WORKERIDS': g['WorkerId'].tolist()}).reset_index(drop=True)
        
        # Store a list of the binary images in the BWS col
        self._df['BWS'] = [data['BinaryImages'].to_list() for group, data in self._grouped_data]

        # given that hash_str is a property of each item in the objects stored in each row of self._df['ImageHashes'], apply self._metatables.item_exists( table_name='IMAGE_HASHES', item_name=hash_str) via list comprehension
        self._df['IS_VALID'] = [self._metatables.item_exists(table_name='IMAGE_HASHES', item_name=item.hash_str) for item in self._df['IMAGE_HASHES']]

        # Create a dicom file for each row such that the semantic segmentation data is stacked
        self._create_DCMDATA()

        # create a list of dataframes such that each dataframe groups together the self._df rows that correspond to images belonging to the same subject.
        self._group_rows_by_subject()
        
        
    def _get_table( self, table_name: str ) -> pd.DataFrame: #to-do: move this to metatables class definition.
        assert self._metatables.table_exists( table_name ), f'Table {table_name} does not exist in metatables'
        return self._metatables.tables[table_name]

    # def _get_item( self, table_name: str, item_name: str ) -> pd.Series: #to-do: move this to metatables class definition
    #     # table name represents the table in self._metatables.tables that we want. item_name is the row of that table that we want. return the row as a pd.series
    #     return self._get_table(table_name).loc[item_name]


    def _read_image( self, s3url: str ) -> ImageHash:
        response = requests.get( s3url, stream=True )
        response.raw.decode_content = True
        arr = np.asarray( bytearray( response.raw.read() ), dtype=np.uint8 )
        return ImageHash( metatables=self._metatables, img=cv2.imdecode( arr, cv2.IMREAD_GRAYSCALE ) ) # img = Image.open( response.raw )

    # write a method that returns the column name containing the substring .pngImageData
    def _get_pngImageData_col( self ) -> str:
        return [c for c in self._raw_data.columns if '.pngImageData' in c][0]

    def convert_base64_to_np_array( self, b64_str: str ) -> np.ndarray:
        return cv2.imdecode( np.frombuffer( base64.b64decode( b64_str ), np.uint8 ), cv2.IMREAD_GRAYSCALE )

    def _derive_subject_instance_num( self, subject_xnat_uid: str, hash_str: str, image_hashes_mt_uid: str ) -> str: # filename that we need to mimic
            
        pass

    def _create_DCMDATA( self ):
        hash_table, subject_table, registered_user_table = self._get_table( 'IMAGE_HASHES' ), self._get_table( 'SUBJECTS' ), self._get_table( 'REGISTERED_USERS' )
        user_mt_uid = registered_user_table.loc[registered_user_table['NAME'] == self._login.validated_username.upper(), 'UID'].values[0]
        for i, row in self._df.iterrows():
            if not row['IS_VALID']:
                continue
            image_hashes_mt_uid = row['IMAGE_HASHES'].hash_str.upper()
            subject_mt_uid = hash_table.loc[hash_table['NAME'] == image_hashes_mt_uid, 'SUBJECT'].values[0] # type: ignore
            self._df.loc[i,'SUBJECT_XNAT_UID'] = subject_table.loc[subject_table['UID'] == subject_mt_uid, 'NAME'].values[0] # type: ignore
            self._df.loc[i,'image_hashes_mt_uid'] = image_hashes_mt_uid
            self._df.loc[i,'subject_mt_uid'] = subject_mt_uid
            dcm = self._create_dicom( subject_name=self._df.loc[i,'SUBJECT_XNAT_UID'], user_uid=user_mt_uid, bws=row['BWS'], uids=row['UID_INFO'] )
            self._df.loc[i, 'DCMDATA'] = [dcm]

    def _create_dicom( self, subject_name: str, user_uid: str, bws: list, uids: dict ) -> pydicomDataset:
        # Create a dicom file to represent the segmentations for the referenced dicom file
        file_meta = pydicomFileMetaDataset()
        file_meta.MediaStorageSOPClassUID = self._mediastoragesopclassuid 
        # file_meta.MediaStorageSOPInstanceUID = pydicomUID.generate_uid()
        # file_meta.ImplementationClassUID = pydicomUID.generate_uid()
        file_meta.TransferSyntaxUID = self._transfersyntaxuid
        
        dcm = pydicom.Dataset()
        dcm.StudyInstanceUID = pydicomUID( subject_name ) # to-do: should be pydicom compliant already, will need to come back to this with a revised metatable
        HIT_id = str( uids['HITIDS'][0] )
        # dcm.UID = pydicomUID( HIT_id ) # HIT Id -- to-do: will need to redo this table to include pydicom compliant uids
        dcm.file_meta = file_meta
        # dcm.ReferencedSOPInstanceUID = original_dcm.SOPInstanceUID
        dcm.InstanceCreationDate, dcm.InstanceCreationTime = self._datetime.date, self._datetime.time
        dcm.InstanceCreatorUID = generate_pydicomUID() # to-do: will need to redo this table to include pydicom compliant uids
        # dcm.InstanceCreatorUID = user_uid # to-do: will need to redo this table to include pydicom compliant uids
        dcm.ImageType = ['DERIVED', 'PRIMARY', 'SEGMENTATION']
        dcm.Modality = 'KO' # Key Object Selection
        dcm.LargestImagePixelValue, dcm.SmallestImagePixelValue = np.uint( 1 ), np.uint( 0 )
        dcm.SeriesDescription = 'Instructions Refinement - Iteration N'
        dcm.SegmentSequence = self._create_segment_sequence( uids, bws )
        return dcm

    def _create_segment_sequence( self, uids: dict, bws: list ) -> Sequence:
        segment_sequence = Sequence()
        for idx, bw in enumerate( bws ):
            seg = Dataset()
            seg.SegmentNumber = idx # Worker ID here
            seg.OperatorsName = str( uids['WORKERIDS'][idx] ) # Worker ID here
            seg.add_new( (0x0019, 0x10a0), 'LO', "Operators' Name : WorkerID" )
            seg.SegmentLabel = str( uids['ASSIGNMENTIDS'][idx] ) # To:do: Assignment ID here to denote the unique submission of the worker for this HIT
            seg.SegmentAlgorithmType, seg.SegmentAlgorithmName, seg.SegmentationType = self._SegmentAlgorithmType, self._SegmentAlgorithmName, self._SegmentationType
            
            # Create a CodeSequence for the SegmentedPropertyCategoryCodeSequence attribute
            code_sequence = Dataset()
            code_sequence.CodeValue, code_sequence.CodingSchemeDesignator, code_sequence.CodeMeaning = self._object_type_code, self._CodingSchemeDesignator, self._object_type
            seg.SegmentedPropertyCategoryCodeSequence = Sequence( [code_sequence] )
            
            # Create a CodeSequence for the SegmentedPropertyTypeCodeSequence attribute -- same as above
            seg.SegmentedPropertyTypeCodeSequence = Sequence( [code_sequence.copy()] )
            
            # seg.SegmentDescription = 'Instructions Refinement - Iteration N'
            seg.PixelData, seg.Rows, seg.Columns = bw.tobytes(), bw.shape[0], bw.shape[1]
            segment_sequence.append( seg )
        #     self._metatables.add_new_item( table_name='MTURK_WORKER_IDS', item_name=seg.SegmentNumber, extra_columns_values={'MTURK_HIT_ID': HIT_id } )
        #     self._metatables.add_new_item( table_name='MTURK_ASSIGNMENT_IDS', item_name=seg.SegmentLabel, extra_columns_values={'MTURK_HIT_ID': HIT_id, 'MTURK_WORKER_ID': seg.SegmentNumber } )
        # self._metatables.add_new_item( table_name='MTURK_HIT_IDS', item_name=HIT_id )
        return segment_sequence
    
    def _group_rows_by_subject( self ):
        pass


    def write( self, idcm: int, zip_dest: Opt[str] = None, print_out: Opt[bool] = False ) -> str: #write individiual dicom files to a zipped folder
        
        if not self._df.loc[idcm, 'IS_VALID']:
            # if print_out:
            print( f'***Session is invalid; could be for several reasons. try evaluating whether all of the image hash_strings already exist in the matatable.' )
        pass

    def catalog_new_data( self ):
        self._metatables.save( print_out=print_out )
        pass

batch_ffn = r'C:\Users\dmattioli\Projects\XNAT\data\examples\MTurkSemanticSegmentation_Example_2.csv'
poo = OtherDicomSession( batch_ffn, validated_login, connection, metatables, object='Humerus' )
